# 제 1고지 미분 자동 계산

In [2]:
import numpy as np 

In [3]:
# Step_1 : 변수
class Variable:
    def __init__(self, data):
        self.data = data

In [5]:
# x 는 데이터 자체가 아니라 데이터를 담는 상자다. 
data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [6]:
x.data = np.array(2.0)
print(x.data)

2.0


In [7]:
# Step_2 : 함수 
# 함수란? : x -> f(x) -> y
# Function 클래스는 Variable 인스턴스를 입력받아 Variable 인스턴스를 출력
# Variable 인스턴스의 실제 데이터는 변수인 data에 있다.abs

class Function : 
    def __call__(self, input) :
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [8]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [10]:
# Function 클래스는 기반 클래스로 모든 함수에 공통되는 기능을 구현하기 위해서
# 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현하기 위해 수정
class Function : 
    def __call__(self, input) :
        x = input.data
        y = self.forward(x) ## 구체적인 계산은 forward 매서드에서 한다
        output = Variable(y)
        return output

    def forward(self, x):
        ## forward 메서드를 직접 호출하면 상속하여 구현해야 한다는 오류를 알려줌 
        raise NotImplementedError() 

In [11]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [14]:
x = Variable(np.array(10))
f = Square()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [15]:
# Step_3 : 함수 연결
# y = exp(x) 의 구현

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [18]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
c = C(b)
print(c.data)

1.648721270700128


In [19]:
# Step_4 : 수치 미분 
# 미분이란? : 변화율 
# ex) 물체의 시간에 따른 위치 변화 = 속도
# ex) 시간에 대한 속도 변화율 = 가속도 
# 정확한 정의 : 극한으로 짧은 시간에서의 변화량 
# f(x)' = lim((f(x+h)-f(x)) / h)  : h->0 

In [22]:
# 미분을 계산하는 코드 구현 
# 컴퓨터는 극한을 취급할 수 없으니 h를 최대한 작은 값을 이용하여 계산 
# 함수의 변화량을 구하는 방법 : 수치 미분 (numerical differentiation) 

# 1e-4 : 0.0001 
def numerical_diff(f, x, eps = 1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [23]:
# y= x**2 에서 x = 2.0 일 때 미분한 결과 
# 4.0 이지만 오차가 발생 // 극한을 표현 할 수 없기 때문 
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [24]:
# 합성 함수 미분 
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)

print(dy)

3.2974426293330694


In [ ]:
# Step_5 : 역전파 이론 
# 수치 미분에는 한계가 있다 -> why?
# 오차가 존재하기 때문 -> 왜 오차가 존재 하는가? 
# 자릿수 누락의 문제가 크다 -> 이를 해결하기 위해 역전파를 씀 

